In [2]:
import pyspark 
from pyspark.sql import SparkSession,Row,DataFrame
import pyspark.sql.functions as F
from pyspark.sql.types import *


spark=SparkSession.builder.master("local[1]")\
        .appName("movie_local")\
        .config("spark.driver.extraClassPath","/Users/eduardoalberto/opt/spark-3.5.3/jars/mysql-connector-j-9.1.0.jar")\
        .config("spark.sql.shuffle.partitions", "7") \
        .config("spark.sql.warehouse.dir", "/Users/eduardoalberto/LoadFile/repository/database/") \
        .config("spark.dynamicAllocation.enabled", "true") \
        .config("spark.executor.cores", "4") \
        .config("spark.dynamicAllocation.minExecutors", "1") \
        .config("spark.dynamicAllocation.maxExecutors", "5") \
        .enableHiveSupport()\
        .getOrCreate()
sc = spark.sparkContext
spark.sparkContext.setLogLevel("OFF") 
print('PySpark Version :'+spark.version)
print('PySpark Version :'+spark.sparkContext.version)

spark

25/01/16 22:06:12 WARN Utils: Your hostname, MacBook-Pro-de-Eduardo.local resolves to a loopback address: 127.0.0.1; using 192.168.3.180 instead (on interface en0)
25/01/16 22:06:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/16 22:06:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


PySpark Version :3.5.4
PySpark Version :3.5.4


In [ ]:
! ls /Users/eduardoalberto/LoadFile/data_2022*.csv



In [4]:
spark.sql(" CREATE DATABASE IF NOT EXISTS dbdev150").show()


++
||
++
++



In [6]:
df = spark.read.option("delimiter", "\t").option("header", True).csv('/Users/eduardoalberto/LoadFile/data_2022*.csv')

df.show(truncate=False)



+---------+-------------------+---------+---------+-------------------------------------+---------------------------------------+
|nconst   |primaryName        |birthYear|deathYear|primaryProfession                    |knownForTitles                         |
+---------+-------------------+---------+---------+-------------------------------------+---------------------------------------+
|nm0000001|Fred Astaire       |1899     |1987     |soundtrack,actor,miscellaneous       |tt0050419,tt0072308,tt0031983,tt0053137|
|nm0000002|Lauren Bacall      |1924     |2014     |actress,soundtrack                   |tt0037382,tt0038355,tt0117057,tt0071877|
|nm0000003|Brigitte Bardot    |1934     |\N       |actress,soundtrack,music_department  |tt0056404,tt0057345,tt0049189,tt0054452|
|nm0000004|John Belushi       |1949     |1982     |actor,soundtrack,writer              |tt0077975,tt0072562,tt0080455,tt0078723|
|nm0000005|Ingmar Bergman     |1918     |2007     |writer,director,actor                |t

In [9]:
import pyspark 
from pyspark.sql import SparkSession,Row,DataFrame
import pyspark.sql.functions as F
from pyspark.sql.types import *

def fn_ingest (df: DataFrame) -> DataFrame :

    dfs = df.filter(" deathYear <> r'\\N' and deathYear <> r'\\N'  ")\
            .withColumn('profession_array',F.explode((F.split(F.col("primaryProfession"), ","))))\
            .withColumn('knownForTitles_array',F.explode((F.split(F.col("knownForTitles"),","))))\
            .drop_duplicates(['profession_array','knownForTitles_array'])

    
    dff = dfs.filter("profession_array <> r'\\N' and knownForTitles_array <> r'\\N'")
  
    return dff
tb = fn_ingest(df)

tb.limit(50000).write.mode("overwrite").saveAsTable("dbdev150.movie")


#tb.limit(50000).write.mode("overwrite").jdbc("jdbc:mysql://localhost:3306/myDbUser", "myDbUser.movie",properties={"user": "root", "password": "mysql"})

#104.466.466
#3.449.518


AnalysisException: [TABLE_OR_VIEW_ALREADY_EXISTS] Cannot create table or view `dbdev150`.`movie` because it already exists.
Choose a different name, drop or replace the existing object, or add the IF NOT EXISTS clause to tolerate pre-existing objects.

In [23]:
df = spark.table("dbdev150.movie")

df.write.mode("overwrite").parquet("/Users/eduardoalberto/LoadFile/parquet/movie")



dfs = spark.read.option("delimiter", ";").option("header", True).csv("/Users/eduardoalberto/LoadFile/part-00000-055103f0-b275-4e27-b667-0c2c25d0636a-c000.csv")
dfs.write.partitionBy("dt_release").mode("overwrite").parquet("/Users/eduardoalberto/LoadFile/parquet/movie02")
# df.write.parquet("/Users/eduardoalberto/LoadFile/parquet/arqui02/")

df.show()

dfs.show()




+----------+-----------+--------------------+--------------------+-----------------+--------------+----------------+--------------------+
|    nconst|primaryName|           birthYear|           deathYear|primaryProfession|knownForTitles|profession_array|knownForTitles_array|
+----------+-----------+--------------------+--------------------+-----------------+--------------+----------------+--------------------+
|tt14495706|      short|   La Rosace Magique|   La Rosace Magique|                0|          1877|               0|                1877|
|tt12592084|      short|   Le singe musicien|   Le singe musicien|                0|          1878|               0|                1878|
|tt15245006|      short|The 1882 Transit ...|The 1882 Transit ...|                0|          1882|               0|                1882|
| tt5459794|      short|     Buffalo Running|     Buffalo Running|                0|          1883|               0|                1883|
| tt8588366|      short|     L'hom